# Assighment #3

Name: Yi Chen<br/>ID: 1001778238

In [1]:
import os
import re
from random import seed, randrange
from math import log

NEG = 0
POS = 1
ALL = 2

In [2]:
def segmentation(str):
    str=str.replace('<br />', '')
    lst = re.sub('[^0-9a-zA-Z]', ' ', str).split()
    return set([x.lower() for x in lst if x])

In [3]:
address=['aclImdb\\train\\neg\\',  'aclImdb\\train\\pos\\', 'aclImdb\\test\\neg\\', 'aclImdb\\test\\pos\\']
files=[os.listdir(x) for x in address]

train_neg, train_pos = [], []
test_neg, test_pos = [], []
data = [train_neg, train_pos, test_neg, test_pos]
label = [0, 1, 0, 1] # 0: neg  1: pos

for i in range(len(address)):
    for file_name in files[i]:
        words = set()
        f=open(address[i]+file_name,'r',encoding='utf8')
        lines=f.readlines()
        for line in lines:
            words = words.union(segmentation(line))
        data[i].append([words,label[i]])

train = train_neg + train_pos
test = test_neg + test_pos

## a. Divide the dataset as train, development and test.

In train set, randomly select 75% of the reviews as training set and 25% as development set

In [4]:
dev=[]

dev_size = int(len(train)*0.25)

seed(1)

for _ in range(dev_size):
    random_index = randrange(len(train))
    dev.append(train.pop(random_index))

print('Size of train set: ', len(train))
print('Size of development set: ', len(dev))
print('Size of test set: ', len(test))

Size of train set:  18750
Size of development set:  6250
Size of test set:  25000


## b. Build a vocabulary

In [5]:
def build_vocabulary(data, occurrence_threshold):
    voca = {} #{'word':[neg_count, pos_count, all_count]}
    
    for rev in data:
        for word in rev[0]:
            if word not in voca:
                voca[word] = [0,0,1]
            else:
                voca[word][ALL]+=1
            voca[word][rev[1]] += 1
    for key in list(voca.keys()):
        if voca[key][ALL]<occurrence_threshold:
            del voca[key]
    return voca

In [6]:
train_voca = build_vocabulary(train, 5)

## c. Calculate the following probability

In [7]:
def count_label(data):
    label_count=[0,0, len(data)] # [neg_count, pos_count, all_count]
    for item in data:
        if item[1]==NEG:
            label_count[NEG]+=1
        else:
            label_count[POS]+=1
    return label_count
        
def cal_occurrence_probability(word, vocabulary, label_count):
    word = word.lower()
    if word not in vocabulary:
        return 0
    return vocabulary[word][ALL]/label_count[ALL]

def cal_conditional_probability(word, vocabulary, label_count, label): 
    word = word.lower()
    if word not in vocabulary:
        return 0
    return vocabulary[word][label]/label_count[label]

In [8]:
label_count_train = count_label(train)
print('P["the"] = ', cal_occurrence_probability('the', train_voca, label_count_train))
print('P["the"|Positive] = ', cal_conditional_probability('the', train_voca, label_count_train, POS))

P["the"] =  0.9911466666666666
P["the"|Positive] =  0.9900683468603161


## d. Calculate accuracy using dev dataset (five fold cross validation)

In [9]:
def predict(review, vocabulary, label_count, lam):
    probability = []
    for label in [NEG, POS]:
        prob = log(label_count[label]/label_count[ALL])
        for word in review[0]:
            if word not in vocabulary:
                continue
            if lam == 0 and vocabulary[word][label]==0:
                prob = 0
                break
            prob += log((vocabulary[word][label]+lam) / (label_count[label]+lam*len(vocabulary)))
        probability.append(prob)
    
    return NEG if probability[NEG]>probability[POS] else POS

def cal_k_fold_accuracy(data, lam, k, occurrence_shreshold):
    accuracy = []

    folds=[]
    fold_size=len(data)//k
    data = list(data) 
    for _ in range(k):
        fold=[]
        for _ in range(fold_size):
            random_index=randrange(len(data))
            fold.append(data.pop(random_index))
        folds.append(fold)
        
    for fold in folds:
        train_fold = list(folds)
        train_fold.remove(fold)
        train_fold = sum(train_fold, [])
        test_fold = fold
        
        voca = build_vocabulary(train_fold, occurrence_shreshold)
        label_count = count_label(train_fold)
        correct = 0
        for review in test_fold:
            if predict(review, voca, label_count, lam) == review[1]:
                correct+=1
        accuracy.append(correct/len(test_fold))
    
    return accuracy

In [10]:
seed(1)

accuracy_without_smoothing_dev = cal_k_fold_accuracy(dev, 0, 5, 5)
print('5-folds accuracy: ', accuracy_without_smoothing_dev)

5-folds accuracy:  [0.6144, 0.652, 0.6208, 0.6296, 0.6352]


## e. Do following experiments

### Compare the effect of Smoothing

In [11]:
def cal_accuracy(train_set, test_set, lam, occurrence_shreshold):
    voca = build_vocabulary(train_set, occurrence_shreshold)
    label_count = count_label(train_set)
    correct = 0
    for review in test_set:
        if predict(review, voca, label_count, lam) == review[1]:
            correct+=1
    return correct/len(test_set)

In [12]:
print("Accuracy when not using Smoothing: ", cal_accuracy(train, dev, 0, 5))
print("Accuracy when using Laplace Estimate: ", cal_accuracy(train, dev, 1, 5))

Accuracy when not using Smoothing:  0.67088
Accuracy when using Laplace Estimate:  0.85136


### Derive Top 10 words that predict positive and negative class

In [13]:
def get_top_10(data, occurrence_shreshold):
    vocabulary = build_vocabulary(data, occurrence_shreshold)
    label_count = count_label(data)
    
    word_predict = {}
    
    for review in data:
        for word in review[0]:
            if word not in vocabulary: continue
            neg_prob = cal_conditional_probability(word, vocabulary, label_count, NEG) * label_count[NEG]/label_count[ALL] \
                         / cal_occurrence_probability(word, vocabulary, label_count) if vocabulary[word][NEG] > 0 else 0
            pos_prob = cal_conditional_probability(word, vocabulary, label_count, POS) * label_count[POS]/label_count[ALL] \
                        / cal_occurrence_probability(word, vocabulary, label_count)  if vocabulary[word][POS] > 0 else 0
            pred = NEG if neg_prob > pos_prob else POS
            if word not in word_predict:
                word_predict[word] = [0,0,0]
            word_predict[word][ALL] += 1
            if pred == review[1]:       
                word_predict[word][pred] += 1
        
    top_neg_lst = sorted(word_predict, key = lambda x: word_predict[x][NEG]/word_predict[x][ALL], reverse=True) [:10]
    top_pos_lst = sorted(word_predict, key = lambda x: word_predict[x][POS]/word_predict[x][ALL], reverse=True)[:10]
    
    return top_neg_lst, top_pos_lst

In [14]:
top_neg_lst, top_pos_lst = get_top_10(dev, 5)
print("Top 10 words that predict negative class: ", top_neg_lst)
print("Top 10 words that predict positive class: ", top_pos_lst)

Top 10 words that predict negative class:  ['gamers', 'incessant', 'humorless', 'keyboard', 'turd', 'mohanlal', 'nisha', 'bachchan', 'kothari', 'sholay']
Top 10 words that predict positive class:  ['din', 'mclaglen', 'fairbanks', 'audiard', 'sunrise', 'tempest', 'finely', 'rory', 'malone', 'sirk']


## f. Using the test dataset

In [15]:
final_accuracy_test = cal_accuracy(train, test, 1, 5) #lambda = 1, occurrence_shreshold = 5
print("Final Accuracy: ", final_accuracy_test)

Final Accuracy:  0.83444
